In [18]:
import pandas as pd

def split_geocoded_cities(df):

    # Handle missing values gracefully
    df['Geocoded_City1'] = df['Geocoded_City1'].fillna('')
    df['Geocoded_City2'] = df['Geocoded_City2'].fillna('')

    # Split the columns based on newlines
    df[['Geocoded_City1_Part1', 'Geocoded_City1_Part2']] = df['Geocoded_City1'].str.split('\n', n=1, expand=True)
    df[['Geocoded_City2_Part1', 'Geocoded_City2_Part2']] = df['Geocoded_City2'].str.split('\n', n=1, expand=True)

    return df

data = pd.read_csv('/content/Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day_20250225.csv')
data = split_geocoded_cities(data)
print(data.head())

#Trim the data set into 2021-2022 flights and 2023 new flights. 2023 new flights are for rout


       tbl  Year  quarter  citymarketid_1  citymarketid_2  \
0  Table 6  2004        2           33851           34027   
1  Table 6  2003        2           30693           34027   
2  Table 6  2004        4           33029           33570   
3  Table 6  2004        2           33198           30424   
4  Table 6  2003        2           30135           30325   

                            city1                             city2  nsmiles  \
0               Oklahoma City, OK    West Palm Beach/Palm Beach, FL     1197   
1                   Nashville, TN    West Palm Beach/Palm Beach, FL      758   
2                     Lincoln, NE                     San Diego, CA     1259   
3                 Kansas City, MO  Wausau/Mosinee/Stevens Point, WI      459   
4  Allentown/Bethlehem/Easton, PA                        Denver, CO     1539   

   passengers    fare  ... lf_ms  fare_low  table_1_flag  \
0          11  242.51  ...  0.51    235.98           0.0   
1         138  136.00  ...  0.73

First we take the data set, which comes from the US department of transportation, and we create data columns that help us. The first step is to split the Geocoded_City into City Name and and latitude and Longitude


In [19]:
import pandas as pd
import re

def split_geocoded_cities(df):
    """
    Splits the 'Geocoded_City1' and 'Geocoded_City2' columns based on newline characters,
    creating new columns for each split part.  Extracts latitude and longitude.

    Args:
      df: The input DataFrame.

    Returns:
      The DataFrame with the new columns.
    """

    # Handle missing values gracefully
    df['Geocoded_City1'] = df['Geocoded_City1'].fillna('')
    df['Geocoded_City2'] = df['Geocoded_City2'].fillna('')

    # Split the columns based on newlines
    df[['Geocoded_City1_Part1', 'Geocoded_City1_Part2']] = df['Geocoded_City1'].str.split('\n', n=1, expand=True)
    df[['Geocoded_City2_Part1', 'Geocoded_City2_Part2']] = df['Geocoded_City2'].str.split('\n', n=1, expand=True)

    # Extract latitude and longitude using regular expressions
    df['latitude1'] = df['Geocoded_City1_Part2'].str.extract(r'(\d+\.\d+)')
    df['longitude1'] = df['Geocoded_City1_Part2'].str.extract(r',\s*(-?\d+\.\d+)')
    df['latitude2'] = df['Geocoded_City2_Part2'].str.extract(r'(\d+\.\d+)')
    df['longitude2'] = df['Geocoded_City2_Part2'].str.extract(r',\s*(-?\d+\.\d+)')

    return df

data = split_geocoded_cities(data)
data.head()

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,...,Geocoded_City2,tbl6pk,Geocoded_City1_Part1,Geocoded_City1_Part2,Geocoded_City2_Part1,Geocoded_City2_Part2,latitude1,longitude1,latitude2,longitude2
0,Table 6,2004,2,33851,34027,"Oklahoma City, OK","West Palm Beach/Palm Beach, FL",1197,11,242.51,...,"West Palm Beach/Palm Beach, FL\n(33.90976, -98...",2.004234e+14,"Oklahoma City, OK","(35.468494, -97.521264)","West Palm Beach/Palm Beach, FL","(33.90976, -98.500854)",35.468494,-97.521264,33.90976,-98.500854
1,Table 6,2003,2,30693,34027,"Nashville, TN","West Palm Beach/Palm Beach, FL",758,138,136.00,...,"West Palm Beach/Palm Beach, FL\n(33.90976, -98...",2.003231e+14,"Nashville, TN","(36.166687, -86.779932)","West Palm Beach/Palm Beach, FL","(33.90976, -98.500854)",36.166687,-86.779932,33.90976,-98.500854
2,Table 6,2004,4,33029,33570,"Lincoln, NE","San Diego, CA",1259,19,193.88,...,"San Diego, CA\n(32.71576, -117.163817)",2.004433e+14,"Lincoln, NE","(40.813599, -96.70261)","San Diego, CA","(32.71576, -117.163817)",40.813599,-96.70261,32.71576,-117.163817
3,Table 6,2004,2,33198,30424,"Kansas City, MO","Wausau/Mosinee/Stevens Point, WI",459,13,199.57,...,"Wausau/Mosinee/Stevens Point, WI\n(38.892062, ...",2.004233e+14,"Kansas City, MO","(39.099792, -94.578559)","Wausau/Mosinee/Stevens Point, WI","(38.892062, -77.019912)",39.099792,-94.578559,38.892062,-77.019912
4,Table 6,2003,2,30135,30325,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,29,272.83,...,"Denver, CO\n(39.738453, -104.984853)",2.003230e+14,"Allentown/Bethlehem/Easton, PA","(40.602753, -75.469759)","Denver, CO","(39.738453, -104.984853)",40.602753,-75.469759,39.738453,-104.984853


Now we create TotalRevenue, as it is a better thing to predict than fare or passenger count, as revenue is more closely related to profit than the other two. TotalRevenue is a hypothetical amount of revenue that could be generated based on a flight between these two cities, which is what we want as an airline

In [21]:
data['TotalRevenue'] = data['fare'] * data['passengers']
data.head()

# Create a dictionary to store city coordinates
city_coordinates = {}

# Example: Print the dictionary to verify or further use it
print(city_coordinates)

# Example: Print the DataFrame to verify missing values are filled
print(data.head())

#remove everything from before 2021 and print head
data = data[data['Year'] >= 2021]
data.head()

#write data into a csv file
data.to_csv('AirlineData.csv', index=False)


{}
           tbl  Year  quarter  citymarketid_1  citymarketid_2  \
9445   Table 6  2024        2           30140           32211   
15683  Table 6  2024        2           30140           32280   
16443  Table 6  2024        2           30140           32337   
16878  Table 6  2024        2           30140           32448   
17477  Table 6  2024        2           30194           30562   

                       city1                  city2  nsmiles  passengers  \
9445         Albuquerque, NM          Las Vegas, NV      486         693   
15683        Albuquerque, NM        Idaho Falls, ID      654          11   
16443        Albuquerque, NM       Indianapolis, IN     1161          71   
16878        Albuquerque, NM  Jackson/Vicksburg, MS      970          15   
17477  Dallas/Fort Worth, TX             Mobile, AL      539          59   

         fare  ...        tbl6pk  Geocoded_City1_Part1  Geocoded_City1_Part2  \
9445   163.12  ...  2.024230e+14                                     

This section of code creates a binary predictor that predicts whether revenue will be below or above average. This is so that we can use binary predictors. It is potentially going to be better to have the total predicted be continuous and have us use regression methods. Ideally in the future I will predict whether the total revenue is greater or lesser than "similar routes". I could use something like KNN to decide which routes are similar to others

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

# Convert 'Year' column to datetime objects
data['Year'] = pd.to_datetime(data['Year'], format='%Y')
print(data.head())
# Create a subset of the data from 2021 onwards
data_recent = data[data['Year'].dt.year >= 2021]

imputer = SimpleImputer(strategy='mean')
data_recent['latitude1'] = imputer.fit_transform(data_recent[['latitude1']])
data_recent['longitude1'] = imputer.fit_transform(data_recent[['longitude1']])
data_recent['latitude2'] = imputer.fit_transform(data_recent[['latitude2']])
data_recent['longitude2'] = imputer.fit_transform(data_recent[['longitude2']])

print(data_recent.head())
# Calculate the average total revenue
average_revenue = data_recent['TotalRevenue'].mean()

# Create a new binary target variable: 1 if TotalRevenue > average, 0 otherwise
data_recent['Revenue_Above_Average'] = (data_recent['TotalRevenue'] > average_revenue).astype(int)

# Prepare the features and target variable
X = data_recent[['nsmiles', 'latitude1', 'longitude1', 'latitude2', 'longitude2']]
y = data_recent['Revenue_Above_Average']

print(X)
# Handle potential missing values (replace with the mean for numerical features)
for col in ['latitude1', 'longitude1', 'latitude2', 'longitude2', 'nsmiles']:
  X[col] = pd.to_numeric(X[col])

imputer = SimpleImputer(strategy='mean') # Create an imputer instance
X = imputer.fit_transform(X) # Impute missing values in X

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

len(data_recent)

# Create and train the SVM model
#svm_model = SVC(kernel='linear') # You can experiment with different kernels
#svm_model.fit(X_train, y_train)

DecisionTreeClassifier = DecisionTreeClassifier()
DecisionTreeClassifier.fit(X_train, y_train)
# Make predictions on the test set
#y_pred = svm_model.predict(X_test)

y_pred = DecisionTreeClassifier.predict(X_test)
# Evaluate the model's accuracy
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Accuracy of the SVM model: {accuracy}")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree Classifier: {accuracy}")

#show confusion matrix for model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


           tbl       Year  quarter  citymarketid_1  citymarketid_2  \
9445   Table 6 2024-01-01        2           30140           32211   
15683  Table 6 2024-01-01        2           30140           32280   
16443  Table 6 2024-01-01        2           30140           32337   
16878  Table 6 2024-01-01        2           30140           32448   
17477  Table 6 2024-01-01        2           30194           30562   

                       city1                  city2  nsmiles  passengers  \
9445         Albuquerque, NM          Las Vegas, NV      486         693   
15683        Albuquerque, NM        Idaho Falls, ID      654          11   
16443        Albuquerque, NM       Indianapolis, IN     1161          71   
16878        Albuquerque, NM  Jackson/Vicksburg, MS      970          15   
17477  Dallas/Fort Worth, TX             Mobile, AL      539          59   

         fare  ...        tbl6pk  Geocoded_City1_Part1  Geocoded_City1_Part2  \
9445   163.12  ...  2.024230e+14          

/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['latitude1']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['longitude1']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['latitude2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['longitude2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
<ipython-input-25-6adde76fd9ca>:35: Settin

array([[42,  7],
       [14,  2]])

Here, for the sake of exhibition, I create a new data point for a hypothetical route from Portland, Maine, to Miami, Florida in order to show how my decision tree classifier works. The decision tree classifier predicts that this route will have below average revenue.

In [26]:
# Create a new data point as a DataFrame
new_data_point = pd.DataFrame({
    'nsmiles': [1354],
    'latitude1': [43.64468],
    'longitude1': [-70.31341],
    'latitude2': [25.79304],
    'longitude2': [-80.27847]
})

# Use the same imputer to transform the new data point
new_data_point_imputed = imputer.transform(new_data_point)

# Make a prediction using the trained DecisionTreeClassifier
predicted_class = DecisionTreeClassifier.predict(new_data_point_imputed)[0]

# Interpret the prediction (assuming 1 means above average, 0 below average)
if predicted_class == 1:
    print("Predicted: Average revenue is above average for this route.")
else:
    print("Predicted: Average revenue is below average for this route.")


Predicted: Average revenue is below average for this route.


In [27]:
# Assuming 'data' DataFrame is already created and processed as in the previous code
data_recent['profit'] = data_recent['TotalRevenue'] - (40 * data_recent['nsmiles']) - 1000

data_recent.head()
#create a new column called profitable that is 1 if profit > 0 and 0 if profit <= 0

data_recent['profitable'] = (data_recent['profit'] > 0).astype(int)

data_recent.head()

#show counts for profitable and not profitable
data_recent['profitable'].value_counts()



,count
profitable,
0,280
1,44


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

# Prepare the features and target variable
X = data_recent[['nsmiles', 'latitude1', 'longitude1', 'latitude2', 'longitude2']]
y = data_recent['profitable'] # Use 'profitable' as the target variable

# Handle potential missing values (replace with the mean for numerical features)
for col in ['latitude1', 'longitude1', 'latitude2', 'longitude2', 'nsmiles']:
  X[col] = pd.to_numeric(X[col], errors='coerce')

imputer = SimpleImputer(strategy='mean') # Create an imputer instance
X = imputer.fit_transform(X) # Impute missing values in X

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a new Decision Tree Classifier for profitability
profitability_tree = DecisionTreeClassifier(min_samples_split = 100)
profitability_tree.fit(X_train, y_train)

# Make predictions
y_pred_profit = profitability_tree.predict(X_test)

# Evaluate accuracy
accuracy_profit = accuracy_score(y_test, y_pred_profit)
print(f"Accuracy of the Decision Tree Classifier for Profitability: {accuracy_profit}")

# Display the confusion matrix
print(confusion_matrix(y_test, y_pred_profit))

# Predict profitability for the new data point
predicted_profitability = profitability_tree.predict(new_data_point_imputed)[0]
print(predicted_profitability)
if predicted_profitability == 1:
    print("Predicted: The route is predicted to be profitable.")
else:
    print("Predicted: The route is predicted to be unprofitable.")


Accuracy of the Decision Tree Classifier for Profitability: 0.8615384615384616
[[55  0]
 [ 9  1]]
0
Predicted: The route is predicted to be unprofitable.


<ipython-input-28-17f30603c334>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce')


IF YOU HAVE AIRLINEDATA.CSV START HERE


In [29]:
#Load AirlineData.csv into a data frame called data
import pandas as pd
data = pd.read_csv('/content/AirlineData.csv')

#For now 40 is a variable that estimates cost per mile for things like fuel and 1000 is a constant for the other costs (labor) that apply to every flight
data['profit'] = data['TotalRevenue'] - (40 * data['fare']) - 1000
data['profitable'] = (data['profit'] > 0).astype(int)
data.head()

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,...,Geocoded_City1_Part2,Geocoded_City2_Part1,Geocoded_City2_Part2,latitude1,longitude1,latitude2,longitude2,TotalRevenue,profit,profitable
0,Table 6,2024,2,30140,32211,"Albuquerque, NM","Las Vegas, NV",486,693,163.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113042.16,105517.36,1
1,Table 6,2024,2,30140,32280,"Albuquerque, NM","Idaho Falls, ID",654,11,362.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3983.87,-11502.93,0
2,Table 6,2024,2,30140,32337,"Albuquerque, NM","Indianapolis, IN",1161,71,296.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21036.59,8184.99,1
3,Table 6,2024,2,30140,32448,"Albuquerque, NM","Jackson/Vicksburg, MS",970,15,281.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4220.25,-8033.75,0
4,Table 6,2024,2,30194,30562,"Dallas/Fort Worth, TX","Mobile, AL",539,59,343.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20237.00,5517.00,1


In [30]:

# Assuming 'data_recent' DataFrame and other necessary variables are already defined as in the provided code.

#import simple imputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# Prepare the features and target variable
X = data[['nsmiles', 'latitude1', 'longitude1', 'latitude2', 'longitude2']]
y = data['profitable']  # Use 'profitable' as the target variable

# Handle potential missing values (replace with the mean for numerical features)
for col in ['latitude1', 'longitude1', 'latitude2', 'longitude2', 'nsmiles']:
    X[col] = pd.to_numeric(X[col], errors='coerce')

imputer = SimpleImputer(strategy='mean')  # Create an imputer instance
X = imputer.fit_transform(X)  # Impute missing values in X

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a new Decision Tree Classifier for profitability
profitability_tree = DecisionTreeClassifier() # You can add hyperparameters here if needed
profitability_tree.fit(X_train, y_train)

y_pred = profitability_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree Classifier: {accuracy}")

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))
f1_score(y_test, y_pred)

Accuracy of the Decision Tree Classifier: 0.5692307692307692
[[24 12]
 [16 13]]


<ipython-input-30-6575a7717869>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce')
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['latitude1' 'longitude1' 'latitude2' 'longitude2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


0.48148148148148145

In [31]:
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sudalairajkumar/daily-temperature-of-major-cities")
print("Path to dataset files:", path)

# Find the CSV file within the downloaded directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        file_path = os.path.join(path, filename)
        break  # Assuming there's only one CSV file

# Create a DataFrame
df = pd.read_csv(file_path)
df.head()


Path to dataset files: /root/.cache/kagglehub/datasets/sudalairajkumar/daily-temperature-of-major-cities/versions/1


<ipython-input-31-61b95e462892>:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [32]:
# Filter for US cities
us_cities = df[df['Country'] == 'US']

# Group by city and calculate the average temperature
city_avg_temps = us_cities.groupby('City')['AvgTemperature'].mean().reset_index()

# Rename the column
city_avg_temps = city_avg_temps.rename(columns={'AvgTemperature': 'AvgTemp'})
city_avg_temps


,City,AvgTemp
0,Abilene,64.236348
1,Akron Canton,50.391355
2,Albany,49.042957
3,Albuquerque,57.692822
4,Allentown,52.194323
...,...,...
149,Wilkes Barre,49.914798
150,Wilmington,54.504608
151,Yakima,51.185742
152,Youngstown,49.573297


In [33]:

import pandas as pd

def trim_city_names(df):
    """Trims city names to everything before the first slash or comma."""
    df['Ncity1'] = df['city1'].str.split('/', n=1, expand=True)[0].str.split(',', n=1, expand=True)[0]
    df['Ncity2'] = df['city2'].str.split('/', n=1, expand=True)[0].str.split(',', n=1, expand=True)[0]
    return df

data = trim_city_names(data)
print(data.head())


       tbl  Year  quarter  citymarketid_1  citymarketid_2  \
0  Table 6  2024        2           30140           32211   
1  Table 6  2024        2           30140           32280   
2  Table 6  2024        2           30140           32337   
3  Table 6  2024        2           30140           32448   
4  Table 6  2024        2           30194           30562   

                   city1                  city2  nsmiles  passengers    fare  \
0        Albuquerque, NM          Las Vegas, NV      486         693  163.12   
1        Albuquerque, NM        Idaho Falls, ID      654          11  362.17   
2        Albuquerque, NM       Indianapolis, IN     1161          71  296.29   
3        Albuquerque, NM  Jackson/Vicksburg, MS      970          15  281.35   
4  Dallas/Fort Worth, TX             Mobile, AL      539          59  343.00   

   ... Geocoded_City2_Part2  latitude1  longitude1 latitude2  longitude2  \
0  ...                  NaN        NaN         NaN       NaN         NaN   


In [34]:
def add_avg_temp(data, city_avg_temps):

    # Create a dictionary for faster lookup
    city_temps_dict = dict(zip(city_avg_temps['City'], city_avg_temps['AvgTemp']))

    # Add the 'avg_temp' column to the DataFrame
    data['avg_temp'] = data['Ncity1'].map(city_temps_dict)
    data['avg_temp2'] = data['Ncity2'].map(city_temps_dict)

    return data

# Example usage (assuming 'data' and 'city_avg_temps' are already defined):
data = add_avg_temp(data, city_avg_temps)
print(data.head())


       tbl  Year  quarter  citymarketid_1  citymarketid_2  \
0  Table 6  2024        2           30140           32211   
1  Table 6  2024        2           30140           32280   
2  Table 6  2024        2           30140           32337   
3  Table 6  2024        2           30140           32448   
4  Table 6  2024        2           30194           30562   

                   city1                  city2  nsmiles  passengers    fare  \
0        Albuquerque, NM          Las Vegas, NV      486         693  163.12   
1        Albuquerque, NM        Idaho Falls, ID      654          11  362.17   
2        Albuquerque, NM       Indianapolis, IN     1161          71  296.29   
3        Albuquerque, NM  Jackson/Vicksburg, MS      970          15  281.35   
4  Dallas/Fort Worth, TX             Mobile, AL      539          59  343.00   

   ... longitude1  latitude2  longitude2 TotalRevenue     profit  profitable  \
0  ...        NaN        NaN         NaN    113042.16  105517.36        

In [35]:
#import simple imputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# Prepare the features and target variable
X = data[['nsmiles', 'latitude1', 'longitude1', 'latitude2', 'longitude2', 'avg_temp', 'avg_temp2']]
y = data['profitable']  # Use 'profitable' as the target variable

# Handle potential missing values (replace with the mean for numerical features)
for col in ['latitude1', 'longitude1', 'latitude2', 'longitude2', 'nsmiles', 'avg_temp', 'avg_temp2']:
    X[col] = pd.to_numeric(X[col], errors='coerce')

imputer = SimpleImputer(strategy='mean')  # Create an imputer instance
X = imputer.fit_transform(X)  # Impute missing values in X

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a new Decision Tree Classifier for profitability
profitability_tree = DecisionTreeClassifier()
profitability_tree.fit(X_train, y_train)

y_pred = profitability_tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree Classifier: {accuracy}")

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#print f1 score of the model
print(f1_score(y_test, y_pred))
print(cm)

Accuracy of the Decision Tree Classifier: 0.47692307692307695
0.45161290322580644
[[17 19]
 [15 14]]


<ipython-input-35-fe02734e581e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce')
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['latitude1' 'longitude1' 'latitude2' 'longitude2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
